In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Set up methods' colors
methods_colors = {
    "oracle": "#FFFFFF",
    "avg_ens": "#FF7133",
    "best_ms": "#33D4FF",
    "detectors": "#CCCCCC",
    "feature_based": "#91AAC2",
    "sit": "#FFB522",
    "conv": "#4494FF",
    "conv_2": "#0048FF",
    "rocket": "#EA7DFF",
    "best_ad_train": "#228B22",
    "worst_ad_test": "#8B0000",
}

### **Make sure to change to the name of the file that you want to read from**

In [4]:
result = pd.read_csv('../../results/execution_time/all_inference_time.csv')
if 'Unnamed: 0' in result.columns:
    result = result.drop(labels='Unnamed: 0', axis=1)
    
result_info = pd.read_csv('../../results/accuracy/all_accuracy_VUS_PR.csv')[['data_len']]
if 'Unnamed: 0' in result_info.columns:
    result_info = result_info.drop(labels='Unnamed: 0', axis=1)

result_all = pd.concat([result, result_info], axis=1).dropna()
result_all = result_all.loc[result_all['data_len'] < 300000]
result_all = result_all.rename(columns={"Avg Ens.": "Avg Ens"})

In [5]:
all_length = [16, 32, 64, 128, 256, 512, 768, 1024]

# Uncomment this command and all plt.savefig(...) commands to save the generated figures
# figure_path = 'dummy/path/{}.svg'

methods_conv = [
    'inception_time_{}',
    'convnet_{}',
    'resnet_{}',]

methods_sit = [
    'sit_conv_{}',
    'sit_linear_{}',
    'sit_stem_{}',
    'sit_stem_relu_{}',]

methods_ts = ['rocket_{}']

methods_classical = [
    'ada_boost_{}',
    'knn_{}',
    'decision_tree_{}',
    'random_forest_{}',
    'mlp_{}',
    'bayes_{}',
    'qda_{}',
    'svc_linear_{}']

old_method = ['IFOREST', 'LOF', 'MP', 'NORMA', 'IFOREST1', 'HBOS', 'OCSVM',
       'PCA', 'AE', 'CNN', 'LSTM', 'POLY']

methods_ens = [meth.format(length) for meth in methods_conv for length in all_length]
methods_ens += [meth.format(length) for meth in methods_sit for length in all_length]
methods_ens += [meth.format(length) for meth in methods_ts for length in all_length]
methods_ens += [meth.format(length) for meth in methods_classical for length in all_length]

In [6]:
list(result_all.columns)

In [7]:
def get_avg_exec(method, bins):
    all_means = []
    all_stds = []
    for i in range(len(bins)-1):
        all_vals = list(result_all.loc[result_all['data_len'] < bins[i+1]].loc[result_all['data_len'] > bins[i]][method].values)
        mean_vals = np.mean(all_vals)
        std_vals = np.std(all_vals)
        
        all_means.append(mean_vals)
        all_stds.append(std_vals)
        
    return all_means, all_stds

In [8]:
bins = [0, 5000, 10000, 20000, 30000, 40000, 50000, 70000, 100000, 150000, 200000, 300000]
bins_pos = [(bins[i] + bins[i+1]) / 2 for i in range(len(bins)-1)]

plt.figure(figsize=(5, 5))

for bin in bins:
    plt.axvline(bin, color='black',linestyle='dotted')

# Plot Detectors
for methods in old_method:
    means, stds = get_avg_exec(method=methods, bins=bins)
    plt.plot(bins_pos, means, color=methods_colors['detectors'])
    plt.scatter(bins_pos, means, color=methods_colors['detectors'], marker='.')
    #plt.fill_between(bins_pos,np.array(means) - np.array(stds),np.array(means) + np.array(stds))

# Plot Convolutional-based
for methods in methods_conv:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        plt.plot(bins_pos, means, color=methods_colors['conv'])
        plt.scatter(bins_pos, means, color=methods_colors['conv'], marker='o')
        
# Plot Transformer-based
for methods in methods_sit:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        plt.plot(bins_pos, means, color=methods_colors['sit'], alpha=0.5)
        plt.scatter(bins_pos, means, color=methods_colors['sit'], marker='s', alpha=0.5)
        
# Plot Rocket
for methods in methods_ts:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        plt.plot(bins_pos, means, color=methods_colors['rocket'], alpha=0.5)
        plt.scatter(bins_pos, means, color=methods_colors['rocket'], marker='x', alpha=0.5)

# Plot Feature-based
for methods in methods_classical:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        plt.plot(bins_pos, means, color=methods_colors['feature_based'], alpha=0.5)
        plt.scatter(bins_pos, means, color=methods_colors['feature_based'], marker='x', alpha=0.5)

# Plot Average Ensembling
means, stds = get_avg_exec(method='Avg Ens', bins=bins)
plt.plot(bins_pos, means, color=methods_colors['avg_ens'])
plt.scatter(bins_pos, means, color=methods_colors['avg_ens'])

# Figure set up
plt.ylabel('Detection time (sec)')
plt.xlabel('Time series length')
plt.yscale('log')
plt.xscale('log')
plt.show()

In [9]:
bins = [0, 5000, 10000, 20000, 30000, 40000, 50000, 70000, 100000, 150000, 200000, 300000]
bins_pos = [(bins[i] + bins[i+1]) / 2 for i in range(len(bins)-1)]

plt.figure(figsize=(5, 5))

for bin in bins:
    plt.axvline(bin, color='black',linestyle='dotted')

# Plot Detectors
for methods in old_method:
    means, stds = get_avg_exec(method=methods, bins=bins)
    plt.plot(bins_pos, means, color=methods_colors['detectors'])
    plt.scatter(bins_pos, means, color=methods_colors['detectors'], marker='.')
    #plt.fill_between(bins_pos,np.array(means) - np.array(stds),np.array(means) + np.array(stds))

# Plot Convolutional-based
for methods in methods_conv:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        plt.plot(bins_pos, means, color=methods_colors['conv'])
        plt.scatter(bins_pos, means, color=methods_colors['conv'], marker='o')
        
# Plot Transformer-based
for methods in methods_sit:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        plt.plot(bins_pos, means, color=methods_colors['sit'], alpha=0.5)
        plt.scatter(bins_pos, means, color=methods_colors['sit'], marker='s', alpha=0.5)
        
# Plot Rocket
for methods in methods_ts:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        plt.plot(bins_pos, means, color=methods_colors['rocket'], alpha=0.5)
        plt.scatter(bins_pos, means, color=methods_colors['rocket'], marker='x', alpha=0.5)

# Plot Feature-based
for methods in methods_classical:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        plt.plot(bins_pos, means, color=methods_colors['feature_based'], alpha=0.5)
        plt.scatter(bins_pos, means, color=methods_colors['feature_based'], marker='x', alpha=0.5)

# Plot Average Ensembling
means, stds = get_avg_exec(method='Avg Ens', bins=bins)
plt.plot(bins_pos, means, color=methods_colors['avg_ens'])
plt.scatter(bins_pos, means, color=methods_colors['avg_ens'])

# Figure set up
plt.ylabel('Detection time (sec)')
plt.xlabel('Time series length')
# plt.yscale('log')
# plt.xscale('log')
plt.show()

In [10]:
plt.figure(figsize=(5, 5))

plt.rcParams.update({'font.size': 15})

# Plot Detectors
all_means = []
for methods in old_method:
    means, stds = get_avg_exec(method=methods, bins=bins)
    all_means.append(means)
plt.plot(bins_pos, np.mean(all_means, 0), color=methods_colors['detectors'])
#plt.fill_between(bins_pos, np.min(all_means, 0), np.max(all_means, 0), color=methods_colors['detectors'], alpha=0.2)
plt.scatter(bins_pos, np.mean(all_means, 0), color=methods_colors['detectors'], marker='.', s=50)

# Plot Convolutional-based
all_means = []
for methods in methods_conv:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        all_means.append(means)
plt.plot(bins_pos, np.mean(all_means, 0), color=methods_colors['conv'])
#plt.fill_between(bins_pos, np.min(all_means, 0), np.max(all_means, 0), color=methods_colors['conv'], alpha=0.2)
plt.scatter(bins_pos, np.mean(all_means, 0), color=methods_colors['conv'], marker='o', s=50)
        
# Plot Transformer-based
all_means = []
for methods in methods_sit:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        all_means.append(means)
plt.plot(bins_pos, np.mean(all_means, 0), color=methods_colors['sit'], alpha=1)
#plt.fill_between(bins_pos, np.min(all_means, 0), np.max(all_means, 0), color=methods_colors['sit'], alpha=0.2)
plt.scatter(bins_pos, np.mean(all_means, 0), color=methods_colors['sit'], marker='s', alpha=1, s=50)

# Plot Rocket
all_means = []
for methods in methods_ts:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        all_means.append(means)
plt.plot(bins_pos, np.mean(all_means, 0), color=methods_colors['rocket'], alpha=0.5)
#plt.fill_between(bins_pos, np.min(all_means, 0), np.max(all_means, 0), color=methods_colors['rocket'], alpha=0.2)
plt.scatter(bins_pos, np.mean(all_means, 0), color=methods_colors['rocket'], marker='x', alpha=0.5, s=50)

# Plot Feature-based
all_means = []        
for methods in methods_classical:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        all_means.append(means)
plt.plot(bins_pos, np.mean(all_means, 0), color=methods_colors['feature_based'], alpha=1)
#plt.fill_between(bins_pos, np.min(all_means, 0), np.max(all_means, 0), color=methods_colors['feature_based'], alpha=0.2)
plt.scatter(bins_pos, np.mean(all_means, 0), color=methods_colors['feature_based'], marker='^', alpha=1, s=50)

# Plot Average Ensembling
means, stds = get_avg_exec(method='Avg Ens', bins=bins)
plt.plot(bins_pos, means, color=methods_colors['avg_ens'])
plt.scatter(bins_pos, means, color='white', s=50, edgecolors=methods_colors['avg_ens'])

# Figure set up     
#plt.ylabel('Detection time (sec)')
#plt.xlabel('Time series length')
plt.yscale('log')
plt.xscale('log')

plt.tight_layout()
# plt.savefig(figure_path.format("7_scalability_test_1"))
plt.show()
plt.close()

In [11]:
plt.figure(figsize=(5, 5))

plt.rcParams.update({'font.size': 15})

# Plot Detectors
all_means = []
for methods in old_method:
    means, stds = get_avg_exec(method=methods, bins=bins)
    all_means.append(means)
plt.plot(bins_pos, np.mean(all_means, 0), color=methods_colors['detectors'])
#plt.fill_between(bins_pos, np.min(all_means, 0), np.max(all_means, 0), color=methods_colors['detectors'], alpha=0.2)
plt.scatter(bins_pos, np.mean(all_means, 0), color=methods_colors['detectors'], marker='.', s=50)

# Plot Convolutional-based
all_means = []
for methods in methods_conv:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        all_means.append(means)
plt.plot(bins_pos, np.mean(all_means, 0), color=methods_colors['conv'])
#plt.fill_between(bins_pos, np.min(all_means, 0), np.max(all_means, 0), color=methods_colors['conv'], alpha=0.2)
plt.scatter(bins_pos, np.mean(all_means, 0), color=methods_colors['conv'], marker='o', s=50)
        
# Plot Transformer-based
all_means = []
for methods in methods_sit:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        all_means.append(means)
plt.plot(bins_pos, np.mean(all_means, 0), color=methods_colors['sit'], alpha=1)
#plt.fill_between(bins_pos, np.min(all_means, 0), np.max(all_means, 0), color=methods_colors['sit'], alpha=0.2)
plt.scatter(bins_pos, np.mean(all_means, 0), color=methods_colors['sit'], marker='s', alpha=1, s=50)

# Plot Rocket
all_means = []
for methods in methods_ts:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        all_means.append(means)
plt.plot(bins_pos, np.mean(all_means, 0), color=methods_colors['rocket'], alpha=0.5)
#plt.fill_between(bins_pos, np.min(all_means, 0), np.max(all_means, 0), color=methods_colors['rocket'], alpha=0.2)
plt.scatter(bins_pos, np.mean(all_means, 0), color=methods_colors['rocket'], marker='x', alpha=0.5, s=50)

# Plot Feature-based
all_means = []        
for methods in methods_classical:
    for length in all_length:
        means, stds = get_avg_exec(method=methods.format(length), bins=bins)
        all_means.append(means)
plt.plot(bins_pos, np.mean(all_means, 0), color=methods_colors['feature_based'], alpha=1)
#plt.fill_between(bins_pos, np.min(all_means, 0), np.max(all_means, 0), color=methods_colors['feature_based'], alpha=0.2)
plt.scatter(bins_pos, np.mean(all_means, 0), color=methods_colors['feature_based'], marker='^', alpha=1, s=50)

# Plot Average Ensembling
means, stds = get_avg_exec(method='Avg Ens', bins=bins)
plt.plot(bins_pos, means, color=methods_colors['avg_ens'])
plt.scatter(bins_pos, means, color='white', s=50, edgecolors=methods_colors['avg_ens'])

# Figure set up     
plt.ylabel('Detection time (sec)')
plt.xlabel('Time series length')    
plt.xticks([0, 50000, 100000, 150000, 200000, 250000], ['0', '50k', '100k', '150k', '200k', '250k'])

plt.tight_layout()
# plt.savefig(figure_path.format("7_scalability_test_2"))
plt.show()
plt.close()